In [55]:
from easygui import fileopenbox
from librosa import load
import os
import winsound

## Selecting the audio file manually

---

In [207]:
default_path_to_dataset = os.path.abspath(os.path.join("Datasets","RAVDESS_DATASET"))
choosen_path = fileopenbox(msg="Choose a voice sample",title="Test Audio Selector",default=default_path_to_dataset,filetypes="*.wav")

## Play The Selected Audio

In [208]:
winsound.PlaySound(choosen_path,winsound.SND_FILENAME)

# Extract The Original Emotion Details from File Name
---

In [209]:
emotion_dictionary = {
        1:'neutral',
        2:'calm',
        3:'happy',
        4:'sad',
        5:'angry',
        6:'fearful',
        7:'disgust',
        8:'surprised'
    }

In [210]:
def find_emotion(filename):
    """
        @filename : taking the voice input filename 
        
        @returning emotion code by extracting it 
    """
    parta,_ = filename.split('.')
    codes = parta.split('-')
    #print(*codes)
    return int(codes[2])

In [211]:
file_name = choosen_path.split("\\")[-1]
original_emotion = find_emotion(file_name)
print(emotion_dictionary[original_emotion])

sad


## Encoding Categories
---

In [212]:
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import load_model

In [213]:
categories = np.array(list(emotion_dictionary.values())).reshape(-1,1)
print(categories)
Ohe = OneHotEncoder(handle_unknown="ignore")
v = Ohe.fit_transform(categories).toarray()
Ohe.get_feature_names_out()

[['neutral']
 ['calm']
 ['happy']
 ['sad']
 ['angry']
 ['fearful']
 ['disgust']
 ['surprised']]


array(['x0_angry', 'x0_calm', 'x0_disgust', 'x0_fearful', 'x0_happy',
       'x0_neutral', 'x0_sad', 'x0_surprised'], dtype=object)

In [214]:
print(v)

[[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


# Extracting The Features of the selected Audio 
-----

### Feature Extraction Function

----

In [215]:
import librosa
import numpy as np

In [216]:
def extract_features(file_path):
    data,sample_rate = load(file_path,duration = 2.5) 
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr)) # stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft)) # stacking horizontally

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally
    
    return np.array(result)

In [217]:
from sklearn.preprocessing import StandardScaler

In [218]:
aud_features = np.array(extract_features(choosen_path)).reshape(-1,1)
scaler = StandardScaler()
aud_features = scaler.fit_transform(aud_features)
aud_features =aud_features.reshape(1,-1,1)
aud_features
print(aud_features.shape)

(1, 162, 1)


# Select the model you want to use
----

In [219]:
model_path = fileopenbox(default=os.path.abspath("Models"),filetypes="\*.h5")

In [220]:
model = load_model(model_path)

In [221]:
model.input_shape

(None, 162, 1)

In [222]:
ans = model.predict(aud_features)
ans

array([[0.0000000e+00, 6.2045796e-21, 0.0000000e+00, 1.5442910e-33,
        0.0000000e+00, 1.6527989e-26, 1.0000000e+00, 0.0000000e+00]],
      dtype=float32)

In [223]:
ans.shape

(1, 8)

In [224]:
Ohe.inverse_transform(ans)[0][0]

'sad'

In [225]:
emotion_dictionary[original_emotion]

'sad'

In [226]:
Ohe.inverse_transform(ans)[0][0] == emotion_dictionary[original_emotion]

True